## Exploratory Analysis - Tests

### Global Imports

In [21]:
### Imports
import os
import json
import pandas as pd

In [6]:
# pip install convokit

### Parameters and Global Variables

In [14]:
# Get current working directory
cwd = os.getcwd()
LOCAL_PATH = cwd.replace("\\", "/")
LOCAL_PATH = LOCAL_PATH.replace("data/statistics", "data/convokit")
LOCAL_PATH

'c:/Users/fdmol/Desktop/MSCAPP/CAPP30254/supreme-court-ml-predictions/supreme_court_predictions/data/convokit'

### Functions

In [15]:
"""
This file serves as the client for convokit.
"""
from convokit import Corpus, download


def get_data():
    """
    Loads and outputs the Supreme Court Corpus data
    """

    print("Loading Supreme Court Corpus Data...")
    corpus = Corpus(filename=download("supreme-corpus"))
    corpus.dump(
        "supreme_corpus", base_path=LOCAL_PATH
    )


In [24]:
### Begin reading data
# This is the path to the data

def open_data(file_name):
    """
    Opens the data and returns it as a dictionary

    :param file_name: The name of the file to open
    :return: The data as a dictionary
    """
    path = LOCAL_PATH + f"/supreme_corpus/{file_name}"
    with open(path) as file:
        data = json.load(file)
    return data

### Pipeline and Tests

In [16]:
get_data()

Loading Supreme Court Corpus Data...
Dataset already exists at C:\Users\fdmol\.convokit\downloads\supreme-corpus


In [25]:
# speakers_dict